# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных уже сделана.

Минимальное значение *accuracy*, подходящее для заказчика = 0.75.

## 1. Изучение данных.

In [1]:
#импортируем библиотеки, которые нам потребуются.
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
print(df.info())
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Данная задача относится к класу "Обучение с учителем". Целевым признаком будет являтся столбец "is_ultra". Так как он принимает только два значения, перед нами задача классификации. 

## 2. Разбиваем данные на выборки

In [3]:
df_train, new_df = train_test_split(df, test_size=0.4, random_state=42)
df_valid, df_test = train_test_split(new_df, test_size=0.5, random_state=42)

In [4]:
print(df_train.info())
print()
print(df_valid.info())
print()
print(df_test.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1928 entries, 2369 to 3174
Data columns (total 5 columns):
calls       1928 non-null float64
minutes     1928 non-null float64
messages    1928 non-null float64
mb_used     1928 non-null float64
is_ultra    1928 non-null int64
dtypes: float64(4), int64(1)
memory usage: 90.4 KB
None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 1198 to 1510
Data columns (total 5 columns):
calls       643 non-null float64
minutes     643 non-null float64
messages    643 non-null float64
mb_used     643 non-null float64
is_ultra    643 non-null int64
dtypes: float64(4), int64(1)
memory usage: 30.1 KB
None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 1545 to 283
Data columns (total 5 columns):
calls       643 non-null float64
minutes     643 non-null float64
messages    643 non-null float64
mb_used     643 non-null float64
is_ultra    643 non-null int64
dtypes: float64(4), int64(1)
memory usage: 30.1 KB
None


Разобьем исходные данные на три выборки: 
- тренировочная выборка - 60% от исходной
- валидационая выборка - 20% от исходной
- тестовая выборка - 20% от исходной.

## 3. Исследуйте модели

#### Рассмотрим три модели машинного обучения: "Дерево решений", "Случайный лес" и "Логистическая регрессия".

In [5]:
train_features = df_train.drop('is_ultra', axis=1)
train_target = df_train['is_ultra']
valid_features = df_valid.drop('is_ultra', axis=1)
valid_target = df_valid['is_ultra']

# Выделим признаки и целевой признак для тестовой и валидационной выборок

##### 1.  Дерево решений

In [6]:
for i in range(1, 10):
    tree_model = DecisionTreeClassifier(random_state=42, max_depth=i)
    tree_model.fit(train_features, train_target)
    tree_prediction = tree_model.predict(valid_features)
    tree_accuracy = accuracy_score(valid_target, tree_prediction)
    print('При глубине дерева', i, 'accuracy =', tree_accuracy)
    

При глубине дерева 1 accuracy = 0.7309486780715396
При глубине дерева 2 accuracy = 0.7822706065318819
При глубине дерева 3 accuracy = 0.7916018662519441
При глубине дерева 4 accuracy = 0.7807153965785381
При глубине дерева 5 accuracy = 0.7729393468118196
При глубине дерева 6 accuracy = 0.776049766718507
При глубине дерева 7 accuracy = 0.7807153965785381
При глубине дерева 8 accuracy = 0.7962674961119751
При глубине дерева 9 accuracy = 0.7807153965785381


##### 2. Случайный лес 

In [7]:
for i in range(10, 40, 10):
    for n in range(4,9,2):
        forest_model = RandomForestClassifier(random_state=42, n_estimators=i, max_depth=n)
        forest_model.fit(train_features, train_target)
        forest_prediction = forest_model.predict(valid_features)
        forest_accuracy = accuracy_score(valid_target, forest_prediction)
        print('При количестве деревьев =', i, 'и глубине дерева =', n, 'accuracy =', forest_accuracy)
       

При количестве деревьев = 10 и глубине дерева = 4 accuracy = 0.7993779160186625
При количестве деревьев = 10 и глубине дерева = 6 accuracy = 0.8040435458786936
При количестве деревьев = 10 и глубине дерева = 8 accuracy = 0.8055987558320373
При количестве деревьев = 20 и глубине дерева = 4 accuracy = 0.7900466562986003
При количестве деревьев = 20 и глубине дерева = 6 accuracy = 0.7978227060653188
При количестве деревьев = 20 и глубине дерева = 8 accuracy = 0.7993779160186625
При количестве деревьев = 30 и глубине дерева = 4 accuracy = 0.7900466562986003
При количестве деревьев = 30 и глубине дерева = 6 accuracy = 0.7853810264385692
При количестве деревьев = 30 и глубине дерева = 8 accuracy = 0.8009331259720062


##### 3. Логистическая регрессия

In [8]:
regression_model = LogisticRegression(random_state=42)
regression_model.fit(train_features, train_target)
regression_prediction = regression_model.predict(valid_features)
regression_accuracy = accuracy_score(valid_target, regression_prediction)
print('При Логистической регрессии accuracy =', regression_accuracy)

При Логистической регрессии accuracy = 0.7200622083981337


### Вывод.

Опробовав три разные модели обучения на наших данных приходим к следующим выводам:
- У логистической регрессии наименьшая доля правильных ответов.
- У дерева решений не самая плохая доля правльных ответов, но эта модель склонна к переобучению, особенно при большой глубине дерева.
- Наилучшая доля правильных ответов у случайного леса. При количестве деревье 10 и 30 доля правильных ответов равна 0.8. 

В качестве финальной модели выберем случайный лес с гиперпараметрами: количество деревьев = 10, глубина дерева = 6. Такие параметры выбраны, так как при них будет наименьшее время обучения, но, при этом, доля правильных ответов не уменьшается.

## 4. Проверка модели на тестовой выборке

In [9]:
test_features = df_test.drop('is_ultra', axis=1)
test_target = df_test['is_ultra']

In [10]:
forest_model = RandomForestClassifier(random_state=42, n_estimators=10, max_depth=6)
forest_model.fit(train_features, train_target)
forest_prediction = forest_model.predict(test_features)
forest_accuracy = accuracy_score(test_target, forest_prediction)
print('Accuracy =', forest_accuracy)

Accuracy = 0.8304821150855366


#### Вывод.
Проверив нашу модель на тестовых данных мы получили долю правельных ответов = 83%. Для данной задачи этого процента хватает. 
Возможно мы могли бы увеличить этот процент, если бы обучающая выборка была больше.


## 5. Проверка модели на адекватность

In [11]:
strategies = ['most_frequent', 'stratified', 'uniform', 'prior'] 
for name in strategies:
    dummy_model = DummyClassifier(random_state=42, strategy=name)
    dummy_model.fit(train_features, train_target)
    score = dummy_model.score(train_features, train_target)
    print('Для стратегии', name, 'средняя точность =', score)
    

Для стратегии most_frequent средняя точность = 0.6924273858921162
Для стратегии stratified средняя точность = 0.58298755186722
Для стратегии uniform средняя точность = 0.5036307053941909
Для стратегии prior средняя точность = 0.6924273858921162


## Вывод. 
Проверив наши данныне используя DummyClassifier, мы видим, что значение "accuracy" у нашей модели значительно выше, чем у DummyClassifier. Из этого делаем вывод, что предсказания нашей модели адекватны.